In [1]:
# this is an implementation of https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub


import torch
from tqdm.notebook import tqdm
!pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
!pip install numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.8 MB/s eta 0:00:00


In [4]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Bert_classification/100k_tiki_crawl.xlsx')
df = df[['Label', 'Product Name']]
df.rename(columns={'Product Name': 'product_name', 'Label': 'label'}, inplace= True)
df.dropna(inplace= True)
df.head()

,label,product_name
0,Thiết bị trình chiếu,"Bút Laser, Bút Trình Chiếu Điều Khiển Từ Xa Es..."
1,Thiết bị trình chiếu,Thiết Bị Trình Chiếu Logitech R400 - Hàng Chín...
2,Thiết bị trình chiếu,Thiết bị trình chiếu Logitech R500S - Hàng Chí...
3,Thiết bị trình chiếu,Bút Thiết Bị Trình Chiếu Logitech R500 USB Blu...
4,Thiết bị trình chiếu,Thiết Bị Trình Chiếu Trợ Giảng bằng tia Lazer ...


In [5]:
label_lst = list(df['label'].unique())
label_dict = {}
for i, label in enumerate(label_lst):
  label_dict[label] = i
# label_dict



In [6]:
label_dict

{'Thiết bị trình chiếu': 0,
 'Cao dán': 1,
 'Cân nhà bếp': 2,
 'Phụ kiện teen nữ': 3,
 'Vỏ nhựa xe máy': 4,
 'Dây thắt thời trang nữ': 5,
 'Kẹo cao su': 6,
 'Thiết bị kiểm soát ra vào': 7,
 'Đào tạo lái xe ô tô': 8,
 'Máy xông hơi ướt': 9,
 'Đĩa chà nhám': 10,
 'Vách tắm ( Cabin tắm )': 11,
 'Máy cưa đĩa': 12,
 'Tay co, đẩy thủy lực': 13,
 'Quạt không cánh': 14,
 'Chậu rửa Lavabo âm bàn': 15,
 'Hộp box điện': 16,
 'Máy vặt lông gia cầm': 17,
 'Máy đo độ nhớt': 18,
 'Xe đẩy phục vụ': 19,
 'Bộ điều khiển nhiệt độ': 20,
 'Giày bóng rổ': 21,
 'Cua': 22,
 'Thịt hun khói': 23,
 'Giá, kệ quán cafe': 24,
 'Máy vắt khô': 25,
 'Đệm phím': 26,
 'Bơm thả chìm': 27,
 'Dụng cụ ngư nghiệp': 28,
 'Xi đánh giày': 29,
 'Quả bóng đá': 30,
 'Nồi áp suất điện': 31,
 'Đục gỗ': 32,
 'Hóa chất xử lý chất thải': 33,
 'Bột màu công nghiệp': 34,
 'Phần mềm quản lý khách hàng': 35,
 'Bộ trễ Delay bảo vệ tủ lạnh': 36,
 'Thép vằn': 37,
 'Sản phẩm massage': 38,
 'Cảm biến đo tốc độ': 39,
 'Dụng cụ thi công cáp q

In [7]:
second_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Bert_classification/2nd_page_tiki_crawl.xlsx')
second_df.dropna(inplace= True)
second_df = second_df[['Label', 'Product Name']]
second_df.rename(columns={'Product Name': 'product_name', 'Label': 'label'}, inplace= True)
df = df.append(second_df, ignore_index= True)
len(df.index)

<ipython-input-7-8d05dfd412f9>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(second_df, ignore_index= True)


169100

In [8]:
import operator
dict_of_counts = df['label'].value_counts().to_dict()
sorted_dict = sorted(dict_of_counts.items(), key=operator.itemgetter(1))
print((sorted_dict))

[('Quạt không cánh', 71), ('Bút đánh dấu bề mặt', 71), ('Thiết bị kiểm định ô tô', 72), ('Thiết bị đo áp suất', 72), ('Nhau thai cừu', 72), ('Dụng cụ thể hình khác', 72), ('Gậy leo núi', 72), ('Bơm tay thủy lực', 73), ('Snack', 73), ('Ống thoát hiểm', 73), ('Trạm trộn bê tông', 73), ('Bảng điện tử', 73), ('Pin và phụ kiện', 73), ('Ghim bấm', 73), ('Sạc pin máy ảnh, máy quay', 73), ('Dao rọc giấy', 73), ('Van các loại ( dùng cho bể bơi )', 73), ('Bếp rán', 73), ('Bộ phụ kiện phòng tắm', 73), ('Bồn rửa y tế', 73), ('Vest nam', 73), ('Chìa khóa (Remote) Ôtô', 73), ('Món Cá', 73), ('Bệt Toilet tự động', 73), ('Biển, bảng, đèn thông báo ( Led )', 73), ('Thiết bị siêu thị', 73), ('Máy tạo Ozone công nghiệp', 73), ('Tủ chống ẩm', 73), ('Chăn cho bé', 73), ('Máy dán nhãn tự động', 73), ('Hệ thống xử lý bụi, thông gió', 73), ('Máy hàn ống', 73), ('Ghế xếp Folding', 73), ('Bột đá (CaCO3)', 73), ('Cáp kết nối tivi', 73), ('Máy tháo lắp lốp Ôtô', 73), ('Gioăng cửa', 73), ('Các loại máy khác', 

In [9]:
sampled_df = pd.DataFrame().reindex_like(df)[0:0]

for label in label_lst:
  df_label = df[df['label'] == label].sample(70)
  sampled_df = pd.concat([sampled_df, df_label])



In [10]:
sampled_df['label_index'] = df.label.replace(label_dict)

In [11]:
# sampled_df.to_excel('/content/drive/MyDrive/Colab Notebooks/Bert_classification/sampled_df.xlsx')
sampled_df.head(5)

,label,product_name,label_index
169081,Thiết bị trình chiếu,Bút trình chiếu từ xa Logitech R400 - Không dâ...,0
10,Thiết bị trình chiếu,"Bút trình chiếu laser có điều khiển từ xa, cổn...",0
15,Thiết bị trình chiếu,[Loại Tốt] Bút Chỉ Laser Trình Chiếu Điều Khiể...,0
169089,Thiết bị trình chiếu,Bút Trình Chiếu Văn Phòng Powerpoint Vesine VP...,0
54005,Thiết bị trình chiếu,"Bút Laser, Bút Trình Chiếu Điều Khiển Từ Xa Es...",0


In [12]:
from sklearn.model_selection import train_test_split
df = sampled_df
df.dropna(inplace= True)
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label_index.values,
                                                  test_size=0.15,
                                                  random_state=42,
                                                  stratify=df.label_index.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

# print(X_val[:5])
# print(y_val[:5])
# print(type(X_val))
# print(type(y_val))

# df.groupby(['label', 'label_index', 'data_type']).count()




In [13]:
df.head(5)

,label,product_name,label_index,data_type
169081,Thiết bị trình chiếu,Bút trình chiếu từ xa Logitech R400 - Không dâ...,0,train
10,Thiết bị trình chiếu,"Bút trình chiếu laser có điều khiển từ xa, cổn...",0,train
15,Thiết bị trình chiếu,[Loại Tốt] Bút Chỉ Laser Trình Chiếu Điều Khiể...,0,train
169089,Thiết bị trình chiếu,Bút Trình Chiếu Văn Phòng Powerpoint Vesine VP...,0,train
54005,Thiết bị trình chiếu,"Bút Laser, Bút Trình Chiếu Điều Khiển Từ Xa Es...",0,train


In [14]:
# train_df = df.loc[df['data_type']=='train']
# train_data = list(train_df['product_name'].values)
# # train_df.values

In [15]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
#                                           do_lower_case=True)

# print(len(train_data))
# print(len(train_data) // 1000)

# for i in range(len(train_data) // 1000):



In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].product_name.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].product_name.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label_index.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label_index.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 40
dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

In [19]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    class_accuracy_dict = {}
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label_index in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label_index]
        y_true = labels_flat[labels_flat==label_index]
        # print(f'Class: {label_dict_inverse[label_index]}')
        # print(f'Accuracy: {len(y_preds[y_preds==label_index])}/{len(y_true)}\n')
        class_accuracy_dict[label_dict_inverse[label_index]] = float(len(y_preds[y_preds==label_index]))/float(len(y_true))
    return class_accuracy_dict

In [21]:
import torch

# Kiểm tra xem GPU có sẵn không
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available.")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU.")

GPU is available.


In [22]:
import os
if not os.path.exists('/content/drive/MyDrive/Colab Notebooks/Bert_classification/fineTuned/'):
    os.makedirs('/content/drive/MyDrive/Colab Notebooks/Bert_classification/fineTuned/')

In [27]:

# validation_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Bert_classification/data_output.xlsx')
# validation_df = validation_df[['Label', 'Product Name']]
# validation_df.rename(columns={'Product Name': 'product_name', 'Label': 'label'}, inplace= True)
# validation_df['label'] = validation_df['label'].str.lstrip('_').str.rstrip('_').str.replace('_', ' ')
# validation_df['label_index'] = validation_df.label.replace(label_dict)
# validation_df.head(5)

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
#                                           do_lower_case=True)

# encoded_data_val = tokenizer.batch_encode_plus(
#     validation_df.product_name.values,
#     add_special_tokens=True,
#     return_attention_mask=True,
#     pad_to_max_length=True,
#     max_length=256,
#     return_tensors='pt'
# )

# batch_size = 40
# input_ids_val = encoded_data_val['input_ids']
# attention_masks_val = encoded_data_val['attention_mask']
# labels_val = torch.tensor(validation_df.label_index.values)

# dataset_val = TensorDataset(input_ids_val, attention_masks_val)

# dataloader_validation = DataLoader(dataset_val,
#                                    sampler=SequentialSampler(dataset_val),
#                                    batch_size=batch_size)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [42]:
# predictions = []

# for batch in dataloader_validation:
#   batch = tuple(b.to(device) for b in batch)
#   inputs = {'input_ids':      batch[0],
#           'attention_mask': batch[1]
#           }

#   with torch.no_grad():
#     outputs = model(**inputs)
#   logits = outputs[0]
#   logits = logits.detach().cpu().numpy()
#   predictions.append(logits)
# predictions = np.concatenate(predictions, axis=0)
# predictions = np.argmax(predictions, axis=1).flatten()
# predictions = list(predictions)
# print(len(predictions))




14976


In [45]:
# inv_label_dict = {v: k for k, v in label_dict.items()}
# print(inv_label_dict)


{0: 'Thiết bị trình chiếu', 1: 'Cao dán', 2: 'Cân nhà bếp', 3: 'Phụ kiện teen nữ', 4: 'Vỏ nhựa xe máy', 5: 'Dây thắt thời trang nữ', 6: 'Kẹo cao su', 7: 'Thiết bị kiểm soát ra vào', 8: 'Đào tạo lái xe ô tô', 9: 'Máy xông hơi ướt', 10: 'Đĩa chà nhám', 11: 'Vách tắm ( Cabin tắm )', 12: 'Máy cưa đĩa', 13: 'Tay co, đẩy thủy lực', 14: 'Quạt không cánh', 15: 'Chậu rửa Lavabo âm bàn', 16: 'Hộp box điện', 17: 'Máy vặt lông gia cầm', 18: 'Máy đo độ nhớt', 19: 'Xe đẩy phục vụ', 20: 'Bộ điều khiển nhiệt độ', 21: 'Giày bóng rổ', 22: 'Cua', 23: 'Thịt hun khói', 24: 'Giá, kệ quán cafe', 25: 'Máy vắt khô', 26: 'Đệm phím', 27: 'Bơm thả chìm', 28: 'Dụng cụ ngư nghiệp', 29: 'Xi đánh giày', 30: 'Quả bóng đá', 31: 'Nồi áp suất điện', 32: 'Đục gỗ', 33: 'Hóa chất xử lý chất thải', 34: 'Bột màu công nghiệp', 35: 'Phần mềm quản lý khách hàng', 36: 'Bộ trễ Delay bảo vệ tủ lạnh', 37: 'Thép vằn', 38: 'Sản phẩm massage', 39: 'Cảm biến đo tốc độ', 40: 'Dụng cụ thi công cáp quang', 41: 'Máy tập chạy bộ', 42: 'M

In [46]:
# print(predictions[:5])

# predictions_txt = []
# for prediction in predictions:
#   predictions_txt.append(inv_label_dict[prediction])

# print(predictions_txt[:5])

[1948, 658, 659, 1949, 1872]
['Ghế văn phòng', 'Thước dây', 'Cưa tay', 'Váy, đầm', 'Dụng cụ đo, kiểm tra khác']


In [26]:

validation_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Bert_classification/data_output.xlsx')
validation_df = validation_df[['Label', 'Product Name']]
validation_df.rename(columns={'Product Name': 'product_name', 'Label': 'label'}, inplace= True)
validation_df['label'] = validation_df['label'].str.lstrip('_').str.rstrip('_').str.replace('_', ' ')
validation_df['label_index'] = validation_df.label.replace(label_dict)
validation_df.head(5)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

encoded_data_val = tokenizer.batch_encode_plus(
    validation_df.product_name.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(validation_df.label_index.values)

dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)


def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)
        print(outputs)
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/Bert_classification/fineTuned/checkpoint_epoch_31.pth')


model.load_state_dict(checkpoint['model_state_dict'])

model.to(device)

_, predictions, true_vals = evaluate(dataloader_validation)
# print(_)
print(predictions)
# print(true_vals)
# class_accuracy_dict = accuracy_per_class(predictions, true_vals)
# class_accuracy_dict = sorted(class_accuracy_dict.items(), key= lambda x:x[1])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ber

KeyboardInterrupt: ignored

In [ ]:
preds_flat = np.argmax(predictions, axis=1).flatten()
preds_flat = list(preds_flat)
print(preds_flat)
print(len(preds_flat))
for i, pre in enumerate(preds_flat[:500]):
  print(str(pre) + ' ' + str(true_vals[i]))

In [ ]:
# class_accuracy_dict = accuracy_per_class(predictions, true_vals)
# class_accuracy_dict = sorted(class_accuracy_dict.items(), key= lambda x:x[1])

# import operator
# dict_of_counts = validation_df['label'].value_counts().to_dict()
# sorted_dict = sorted(dict_of_counts.items(), key=operator.itemgetter(1))
# print((sorted_dict))
# new_sorted_dict = {}
# for element in sorted_dict:
#   new_sorted_dict[element[0]] = element[1]

# sorted_dict = new_sorted_dict

# right_labeling = 0
# for label in class_accuracy_dict:
#   print(label[0]+ ' ' + str(label[1]) + ' ' + str(sorted_dict[label[0]]))

In [ ]:
# import random
# import torch

# torch.cuda.is_available()

# torch.cuda.empty_cache()


# seed_val = 17
# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)





# def evaluate(dataloader_val):

#     model.eval()

#     loss_val_total = 0
#     predictions, true_vals = [], []

#     for batch in dataloader_val:

#         batch = tuple(b.to(device) for b in batch)

#         inputs = {'input_ids':      batch[0],
#                   'attention_mask': batch[1],
#                   'labels':         batch[2],
#                  }

#         with torch.no_grad():
#             outputs = model(**inputs)

#         loss = outputs[0]
#         logits = outputs[1]
#         loss_val_total += loss.item()

#         logits = logits.detach().cpu().numpy()
#         label_ids = inputs['labels'].cpu().numpy()
#         predictions.append(logits)
#         true_vals.append(label_ids)

#     loss_val_avg = loss_val_total/len(dataloader_val)

#     predictions = np.concatenate(predictions, axis=0)
#     true_vals = np.concatenate(true_vals, axis=0)

#     return loss_val_avg, predictions, true_vals

# for epoch in tqdm(range(1, epochs+1)):

#     model.train()

#     loss_train_total = 0

#     progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
#     for batch in progress_bar:

#         model.zero_grad()
#         model.to(device)

#         batch = tuple(b.to(device) for b in batch)

#         inputs = {'input_ids':      batch[0],
#                   'attention_mask': batch[1],
#                   'labels':         batch[2],
#                  }

#         outputs = model(**inputs)

#         loss = outputs[0]
#         loss_train_total += loss.item()
#         loss.backward()

#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

#         optimizer.step()
#         scheduler.step()

#         progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

#     torch.save({
#         'model_state_dict': model.state_dict(),
#         'optimizer_state_dict':optimizer.state_dict(),
#         'epoch':epoch
#     }, f'/content/drive/MyDrive/Colab Notebooks/Bert_classification/fineTuned/checkpoint_epoch_{epoch}.pth')


#     tqdm.write(f'\nEpoch {epoch}')

#     loss_train_avg = loss_train_total/len(dataloader_train)
#     tqdm.write(f'Training loss: {loss_train_avg}')

#     val_loss, predictions, true_vals = evaluate(dataloader_validation)
#     val_f1 = f1_score_func(predictions, true_vals)
#     tqdm.write(f'Validation loss: {val_loss}')
#     tqdm.write(f'F1 Score (Weighted): {val_f1}')




In [ ]:
import random
import torch
from transformers import AdamW, get_linear_schedule_with_warmup

torch.cuda.empty_cache()

device = torch.device("cuda")

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

epochs = 10

def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)


# cai nay chi chay 1 lan thoi
checkpoint = torch.load('/content/drive/MyDrive/Colab Notebooks/Bert_classification/fineTuned/checkpoint_epoch_32.pth')


model.load_state_dict(checkpoint['model_state_dict'])

model.to(device)

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# model.load_state_dict(checkpoint['model_state_dict'])




for epoch in tqdm(range(33, 33+epochs+1)):

    model.train()

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        model.to(device)

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict':optimizer.state_dict(),
        'epoch':epoch
    }, f'/content/drive/MyDrive/Colab Notebooks/Bert_classification/fineTuned/checkpoint_epoch_{epoch}.pth')


    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/11 [00:00<?, ?it/s]

Epoch 33:   0%|          | 0/2932 [00:00<?, ?it/s]


Epoch 33
Training loss: 1.0750353554972685
Validation loss: 0.9802504565897595
F1 Score (Weighted): 0.6987404259694828


Epoch 34:   0%|          | 0/2932 [00:00<?, ?it/s]


Epoch 34
Training loss: 1.0275523250193257
Validation loss: 0.9846930810786122
F1 Score (Weighted): 0.6962864612604555


Epoch 35:   0%|          | 0/2932 [00:00<?, ?it/s]

KeyboardInterrupt: ignored